In [67]:
from typing import List,Tuple

from langchain.llms import OpenAI
from langchain.output_parsers import PydanticOutputParser
from langchain.prompts import PromptTemplate
from langchain.pydantic_v1 import BaseModel, Field, validator
from langchain.chat_models import ChatOpenAI
import os
from dotenv import load_dotenv

In [68]:
model_name = "text-davinci-003"
temperature = 0.0
model = OpenAI(model_name=model_name, temperature=temperature,api_key=os.getenv("basit_key"))

In [69]:
class Joke(BaseModel):
    setup: str = Field(description="question to set up a joke")
    punchline: str = Field(description="answer to resolve the joke")

In [70]:
joke_query = "Tell me a joke."
parser = PydanticOutputParser(pydantic_object=Joke)
parser.get_format_instructions()

'The output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"properties": {"setup": {"title": "Setup", "description": "question to set up a joke", "type": "string"}, "punchline": {"title": "Punchline", "description": "answer to resolve the joke", "type": "string"}}, "required": ["setup", "punchline"]}\n```'

In [71]:
prompt=PromptTemplate.from_template(template="Answer the user query.\n{format_instructions}\n{query}\n")


In [72]:
_input=prompt.format_prompt(
    format_instructions=parser.get_format_instructions(),
    query=joke_query
)

In [73]:


output = model(_input.to_string())

parser.parse(output)

Joke(setup='Why did the chicken cross the road?', punchline='To get to the other side!')

In [122]:
class Product(BaseModel):
    # id:int = Field(description="unique id ")
    # productname: str = Field(description="Name of a product")
    # customer_name :str = Field(description="Name of customer")
    problems: str = Field(description="List the customer name and problem with the product")


actor_query = """"I recently purchased this cookingset with high hopes, but unfortunately, it failed to meet my expectations. The device's performance was inconsistent, with fluctuating heat settings that made styling my hair a bit of a challenge. Moreover, the noise level was surprisingly loud, disrupting my morning routine.\
The build quality left much to be desired; the handle felt fragile, I would not recommend this hair dryer based on my experience.\
Emily Smith

I recently purchased this hairdyer with high hopes, but unfortunately, it failed to meet my expectations. The device's performance 
The build quality left much to be desired; the handle felt fragile, I would not recommend this hair dryer based on my experience.\
Basit
"""

parser = PydanticOutputParser(pydantic_object=Product)

prompt = PromptTemplate(
    template="list of reviews is given by the customer is tripple backslash and to separate each review start with another line\
    you have to extract the main cause of problem in 1 words if there is any, along with name and what they purchased : in following format\
       id ,problems \
    ```{query}```\
    {format_instructions}",
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

_input = prompt.format_prompt(query=actor_query)

output = model(_input.to_string())

# print(parser.parse(output))

In [123]:
print(output)



Output:
{"problems": ["Emily Smith - Performance, Build Quality", "Basit - Performance, Build Quality"]}


In [124]:
parser.parse(output)

OutputParserException: Failed to parse Product from completion 

Output:
{"problems": ["Emily Smith - Performance, Build Quality", "Basit - Performance, Build Quality"]}. Got: 1 validation error for Product
problems
  str type expected (type=type_error.str)

In [76]:
parser.parse(output)

Product(productname='Cookingset/Hairdryer', Details=['Emily Smith: Inconsistent performance, Loud noise, Poor build quality', 'Basit: Inconsistent performance, Loud noise, Poor build quality'])

In [203]:

class ProductEntry(BaseModel):
    id:int
    customer_name: str = Field(description="Name of the customer")
    problem: str = Field(description="Problem with the product")
class Review(BaseModel):
    Review: list[ProductEntry]



review = """I recently purchased this cookingset with high hopes, but unfortunately, it failed to meet my expectations. The device's performance was inconsistent, with fluctuating heat settings that made styling my hair a bit of a challenge. Moreover, the noise level was surprisingly loud, disrupting my morning routine.\
The build quality left much to be desired; the handle felt fragile, I would not recommend this hair dryer based on my experience.\
Emily Smith \

I recently purchased this cookingset with fryingpan, but unfortunately, it failed to meet my expectations. The device's performance was inconsistent, with fluctuating heat settings that made styling my hair a bit of a challenge. Moreover, the noise level was surprisingly loud, disrupting my morning routine.\
The build quality left much to be desired; the handle felt fragile, I would not recommend this hair dryer based on my experience.\
Abdul Basit

I recently purchased this generator with highhopes, but unfortunately, it failed to meet my expectations. The device's performance was inconsistent, with fluctuating heat settings that made styling my hair a bit of a challenge. Moreover, the noise level was surprisingly loud, disrupting my morning routine.\
The build quality left much to be desired; the handle felt fragile, I would not recommend this hair dryer based on my experience.\
Muhammad Saqib\
"""

parser = PydanticOutputParser(pydantic_object=Review)

prompt = PromptTemplate(
    template="List of reviews given by customers by tripple backslash. Each review starts on a new line. Extract the main cause of the problem in 2-3 words, along with the name and the purchased product.\n{format_instructions}\n```{query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

_input = prompt.format_prompt(query=review)

output = model(_input.to_string())  # Replace 'model' with your actual OpenAI model instance

# parsed_output = parser.parse(output)
print(type(output))
# print(parsed_output)

<class 'str'>


In [204]:
print(type(output))

<class 'str'>


In [205]:
parsed_output = parser.parse(output)
print(parsed_output.json())

{"ProductName": ["cookingset", "cookingset with fryingpan", "generator"], "Review": [{"id": 1, "customer_name": "Emily Smith", "problem": "inconsistent performance"}, {"id": 2, "customer_name": "Abdul Basit", "problem": "inconsistent performance"}, {"id": 3, "customer_name": "Muhammad Saqib", "problem": "inconsistent performance"}]}


Review(Review=[ProductEntry(id=1, customer_name='Emily Smith', product_name='Cookingset', problem='Inconsistent heat'), ProductEntry(id=2, customer_name='Abdul Basit', product_name='Fryingpan', problem='Inconsistent heat')])

In [157]:
Joke(setup='Why did the chicken cross the road?', punchline='To get to the other side!')


Joke(setup='Why did the chicken cross the road?', punchline='To get to the other side!')

In [135]:
class Productlisted(BaseModel):
    productname: str = Field(description="name of an actor")
    problem: List[str] = Field(description="list of names of films they starred in")




actor_query = "I recently purchased this cookingset with high hopes, but unfortunately, it failed to meet my expectations. The device's performance was inconsistent, with fluctuating heat settings that made styling my hair a bit of a challenge. Moreover, the noise level was surprisingly loud, disrupting my morning routine.\
The build quality left much to be desired; the handle felt fragile, I would not recommend this hair dryer based on my experience.\
Emily Smit"

parser = PydanticOutputParser(pydantic_object=Productlisted)

prompt = PromptTemplate(
    template="List of reviews given by customers by tripple backslash. Each review starts on a new line. Extract the main cause of the problem in 2-3 words, along with the name and the purchased product.\n{format_instructions}\n```{query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

_input = prompt.format_prompt(query=actor_query)

output = model(_input.to_string())



In [136]:
parser.parse(output)

Productlisted(productname='Cookingset', problem=['inconsistent performance', 'fluctuating heat', 'loud noise', 'fragile handle'])